In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from datetime import datetime

warehouse_path = "./warehouse"
iceberg_spark_jar  = 'org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.3.0'
catalog_name = "demo"

# Setup iceberg config
conf = (
    SparkConf()
    .setAppName("EvergreenApp")
    .set(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    )
    .set(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog")
    .set("spark.jars.packages", iceberg_spark_jar)
    .set(f"spark.sql.catalog.{catalog_name}.warehouse", warehouse_path)
    .set(f"spark.sql.catalog.{catalog_name}.type", "hadoop")
    .set("spark.sql.defaultCatalog", catalog_name)
)

sc = SparkSession.builder.master("local").config(conf=conf).getOrCreate()

batch = datetime.now().strftime("%Y%m%d_%H%M%S")
print(batch)

Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=ISO-8859-1
Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=ISO-8859-1
24/02/28 18:41:28 WARN Utils: Your hostname, Erics-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
24/02/28 18:41:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/ericyeung/Library/Caches/pypoetry/virtualenvs/evergreen-DHscqOYO-py3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ericyeung/.ivy2/cache
The jars for the packages stored in: /Users/ericyeung/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5c71d068-3641-436d-a3a1-746a45be8453;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.3.0 in central
:: resolution report :: resolve 119ms :: artifacts dl 5ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.3.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.a

20240228_184133


Steps Iceberg

In [ ]:
from conf.init_schema import init_database

init_database(sc)

Postcode related

In [ ]:
from extract_utils import extract_csv

df_postcode_lookup = extract_csv(sc, "src_data/National_Statistics_Postcode_Lookup_Latest_Centroids.csv")
df_postcode_lookup.printSchema()
df_postcode_lookup.count()

In [ ]:
from transform.postcode import PostcodeTransformer
df_postcode = PostcodeTransformer.transform(df_postcode_lookup)

In [ ]:
df_postcode.show()

In [ ]:
df_postcode.write.mode("overwrite").insertInto(f"{catalog_name}.db.postcode")

Users related

In [2]:
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType

users_schema = StructType(
[StructField("email", StringType(), True),
 StructField("givenName", StringType(), True),
 StructField("familyName", StringType(), True),
 StructField("sex", StringType(), True),
 StructField("dateOfBirth", StringType(), True),
 StructField("address", StructType([
     StructField("street", StringType(), True),
     StructField("postcode", StringType(), True)]), True),]
)

In [3]:
from extract_utils import extract_json

df_users = extract_json(sc, "src_data/users1.json", users_schema)
df_users.count()

3

Data Validation

In [4]:
from validation_utils import check_data_quality

df_validated = check_data_quality(df_users, batch)

2024-02-28 18:41:45,856 - eg-etl - INFO - [validation_utils.py:21 -   check_data_quality() ] - Original dataframe count: 3

2024-02-28 18:41:45,856 - eg-etl - INFO - [validation_utils.py:21 -   check_data_quality() ] - Original dataframe count: 3

2024-02-28 18:41:45,856 - eg-etl - INFO - [validation_utils.py:21 -   check_data_quality() ] - Original dataframe count: 3

+---------------------------+---------+----------+----+-----------+---------------+---+
|email                      |givenName|familyName|sex |dateOfBirth|address        |_id|
+---------------------------+---------+----------+----+-----------+---------------+---+
|Regina80@hotmail.com       |Regina   |Tremblay  |M   |1982-08-09 |{NULL, CV4 9RE}|0  |
|Vickie.Hilpert2@hotmail.com|Vickie   |Hilpert   |F   |1988-02-28 |{NULL, GU342TZ}|1  |
|Gloria69@hotmail.com       |NULL     |NULL      |NULL|NULL       |{NULL, NULL}   |2  |
+---------------------------+---------+----------+----+-----------+---------------+---+

+----------

In [5]:
df_validated.count()

0